In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

from custom_datasets.lightning import HuggingFaceTokenizedDataModule
from transformers import AutoModelForSequenceClassification
import pytorch_lightning as pl
from models.lightning import LitHuggingfaceClassifier

In [3]:
# checkpoint = "google/flan-t5-small"
checkpoint = "distilbert-base-multilingual-cased"
# checkpoint = "bert-base-multilingual-uncased"


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("../../models/distillbert-12-10/distilbert-base-multilingual-cased/")
# config = AutoConfig.from_pretrained(checkpoint, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
batch_size = 8
pl_model = LitHuggingfaceClassifier(checkpoint, learning_rate=1e-5)
data_module = HuggingFaceTokenizedDataModule(checkpoint=checkpoint, batch_size=batch_size)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# pl_model = LitHuggingfaceClassifier.load_from_checkpoint("../../lightning_logs/comments_sentiment/tensorboard/DistilBertForSequenceClassification/version_5/checkpoints/epoch=4-step=31975.ckpt", model=model)

In [16]:
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks import RichProgressBar

tensorboard_logger = TensorBoardLogger(save_dir="../../lightning_logs/comments_sentiment/tensorboard/", name=f"{type(pl_model.model).__name__}")
csv_logger = CSVLogger(save_dir="../../lightning_logs/comments_sentiment/csv/", name=f"{type(pl_model.model).__name__}")

In [17]:
trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=6,
    callbacks=[RichProgressBar()],
    # logger=[tensorboard_logger, csv_logger]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
# model = AutoModelForSequenceClassification.from_pretrained(
#     "../../models/distillbert-12-10/distilbert-base-multilingual-cased/"
# )
# pl_model = LitHuggingfaceClassifier(model, learning_rate=1e-5)


# trainer.fit(
#     model=pl_model,
#     datamodule=data_module
#     # ckpt_path="../../lightning_logs/comments_sentiment/tensorboard/DistilBertForSequenceClassification/version_11/checkpoints/epoch=2-step=34365.ckpt",
# )
# trainer.validate(pl_model, dataloaders=eval_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name            ┃ Type                                ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model           │ DistilBertForSequenceClassification │  135 M │
│ 1 │ train_acc       │ MulticlassAccuracy                  │      0 │
│ 2 │ valid_acc_micro │ MulticlassAccuracy                  │      0 │
│ 3 │ valid_acc_macro │ MulticlassAccuracy                  │      0 │
│ 4 │ test_acc        │ MulticlassAccuracy                  │      0 │
└───┴─────────────────┴─────────────────────────────────────┴────────┘

Trainable params: 135 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 135 M                                                                                                
Total estimated model params size (MB): 541

Output()

/home/kamil/miniconda3/envs/thesis/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector
.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value 
of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/kamil/miniconda3/envs/thesis/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector
.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value
of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.

/home/kamil/miniconda3/envs/thesis/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected 
KeyboardInterrupt, attempting graceful shutdown...

In [19]:
data_module = HuggingFaceTokenizedDataModule(checkpoint=checkpoint, batch_size=batch_size)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [29]:
# model = AutoModelForSequenceClassification.from_pretrained("../../models/distillbert-12-18/distilbert-base-multilingual-cased/")
pl_model = LitHuggingfaceClassifier("../../models/distillbert-12-18/distilbert-base-multilingual-cased/")
trainer.validate(pl_model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      valid_acc_macro      │    0.9097282886505127     │
│      valid_acc_micro      │    0.9083999991416931     │
│        valid_loss         │    0.2593173682689667     │
└───────────────────────────┴───────────────────────────┘

[{'valid_loss': 0.2593173682689667,
  'valid_acc_micro': 0.9083999991416931,
  'valid_acc_macro': 0.9097282886505127}]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("../../models/distillbert-12-10/distilbert-base-multilingual-cased/")
pl_model = LitHuggingfaceClassifier(model, learning_rate=1e-5)
trainer.validate(pl_model, datamodule=data_module)

In [13]:
model.save_pretrained("../../models/distillbert-12-18/distilbert-base-multilingual-cased")